In [69]:
#Importing libraries
import numpy as np
import math as m
import pandas as pd
import matplotlib.pyplot as plt

In [70]:

#Import data from roads
InitialRoadData = pd.read_csv('..\RMMS\CSV\_allRoads.csv')
RoadData = InitialRoadData


In [71]:
#Deleting inconsistent coordinates 
#Finding quantities out of tolerance. This tolerance may be improved by checking average differences!!
Tolerance = 0.02
RoadData['UpperErrorLat']=abs(RoadData['Latitude'].shift(-1)-RoadData['Latitude'])>Tolerance
RoadData['BelowErrorLat']=abs(RoadData['Latitude'].shift()-RoadData['Latitude'])>Tolerance
RoadData['UpperErrorLon']=abs(RoadData['Longitude'].shift(-1)-RoadData['Longitude'])>Tolerance
RoadData['BelowErrorLon']=abs(RoadData['Longitude'].shift()-RoadData['Longitude'])>Tolerance
#Checking Road ID
RoadData['RoadIDCheckAbove']=RoadData['Road Id']==RoadData['Road Id'].shift(-1)
RoadData['RoadIDCheckBelow']=RoadData['Road Id']==RoadData['Road Id'].shift()
#Labeling Errors
RoadData['Error']=((RoadData['RoadIDCheckAbove'] & RoadData['RoadIDCheckBelow'] & ((RoadData['UpperErrorLat'] & RoadData['BelowErrorLat'])|(RoadData['UpperErrorLon'] & RoadData['BelowErrorLon'])))|
                   (RoadData['RoadIDCheckAbove'] & -RoadData['RoadIDCheckBelow'] & ((RoadData['UpperErrorLat'] &  -RoadData['UpperErrorLat'].shift(-1))|(RoadData['UpperErrorLon'] &  -RoadData['UpperErrorLon'].shift(-1))))|
                   (-RoadData['RoadIDCheckAbove'] & RoadData['RoadIDCheckBelow'] & ((RoadData['BelowErrorLat'] &  -RoadData['BelowErrorLat'].shift())|(RoadData['BelowErrorLon'] &  -RoadData['BelowErrorLon'].shift())))
                  )


In [72]:
#Erasing Wrong values
RoadData.loc[RoadData.Error == True, 'Latitude'] = np.NaN
RoadData.loc[RoadData.Error == True, 'Longitude'] = np.NaN      


In [73]:
#Interpolating 
SmoothRoadData = RoadData.interpolate()
#Formatting
del SmoothRoadData['UpperErrorLat']
del SmoothRoadData['UpperErrorLon']
del SmoothRoadData['BelowErrorLat']
del SmoothRoadData['BelowErrorLon']
del SmoothRoadData['RoadIDCheckAbove']
del SmoothRoadData['RoadIDCheckBelow']
del SmoothRoadData['Error']
SmoothRoadData.columns=SmoothRoadData.columns.str.replace('LRP Number','IDnumber')
SmoothRoadData.columns=SmoothRoadData.columns.str.replace('Road Id','RoadNo')



In [74]:
#Load Bridge Data
InitialBridgeData = pd.read_csv('..\BMMS\CSV\_allBridges.csv')
#This texts deletes NaN values
BridgeData = InitialBridgeData
BridgeData.columns=BridgeData.columns.str.replace('BridgeNo','IDnumber')

In [75]:
#Merge Data Frames
TotalData = pd.merge(BridgeData, SmoothRoadData, on=['RoadNo','IDnumber'], how='outer')
TotalData = TotalData.fillna(-1)


In [76]:
#Detecting Errors
#Tolerance of 100m
Tolerance=0.0008993614
TotalData['LatitudeNotMatching']=abs(TotalData.Latitude_x-TotalData.Latitude_y)>Tolerance  
TotalData['LongitudeNotMatching']=abs(TotalData.Longitude_x-TotalData.Longitude_y)>Tolerance
TotalData['NoDataInRoadFile']=TotalData.Latitude_y==-1
TotalData['NoDataInBridgeFile']=TotalData.Latitude_x==-1
TotalData.loc[TotalData['LatitudeNotMatching']&TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='LatAndLongNotMatching'
TotalData.loc[TotalData['LatitudeNotMatching']&-TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='LatNotMatching'
TotalData.loc[-TotalData['LatitudeNotMatching']&TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='LongNotMatching'
TotalData.loc[TotalData['NoDataInRoadFile']&TotalData['NoDataInBridgeFile'],'TypeOfError']='NoDataInRoadAndBridgeFile'
TotalData.loc[TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='NoDataInRoadFile'
TotalData.loc[-TotalData['NoDataInRoadFile']&TotalData['NoDataInBridgeFile'],'TypeOfError']='NoDataInBridgeFile'
TotalData.loc[-TotalData['LatitudeNotMatching']&-TotalData['LongitudeNotMatching']&-TotalData['NoDataInRoadFile']&-TotalData['NoDataInBridgeFile'],'TypeOfError']='CorrectData'




In [77]:
#Format Final File
FinalData=TotalData
del FinalData['LatitudeNotMatching']
del FinalData['LongitudeNotMatching']
del FinalData['NoDataInRoadFile']
del FinalData['NoDataInBridgeFile']
FinalData.columns=FinalData.columns.str.replace('Latitude_x','Latitude_B')
FinalData.columns=FinalData.columns.str.replace('Latitude_y','Latitude_R')
FinalData.columns=FinalData.columns.str.replace('Longitude_x','Longitude_B')
FinalData.columns=FinalData.columns.str.replace('Longitude_y','Longitude_R')


In [82]:
FinalData[FinalData['TypeOfError']=='CorrectData']

,Number,Structure Name,Condition of Bridge,TotalWidth,TotalLength,ConstructionYear,NumberOfSpan,Zone,Circle,Division,...,LocationLRPOffset,LocationChainage,Latitude_B,Longitude_B,Road Chainage,Type,Desc,Latitude_R,Longitude_R,TypeOfError
3,4.0,KATCHPUR BOX CULVERT,B,21.70,8.00,1987.0,2.0,Dhaka,Dhaka,Narayanganj,...,543.0,10.543,23.702056,90.528194,9.615,Culvert,Box culvert,23.701528,90.528194,CorrectData
4,5.0,NOYAPARA CULVERT,A,12.20,6.30,1992.0,2.0,Dhaka,Dhaka,Narayanganj,...,880.0,10.880,23.699833,90.530722,9.963,Culvert,Box culvert,23.699861,90.530722,CorrectData
62,63.0,ILLIOT BAZER,C,9.20,28.90,1985.0,1.0,Comilla,Comilla,Comilla,...,896.0,56.006,23.513194,90.870583,55.333,Bridge,Bridge start,23.513306,90.870305,CorrectData
68,69.0,MILL GHAT BOX CULVERT,A,8.30,4.50,1991.0,1.0,Comilla,Comilla,Comilla,...,940.0,68.040,23.495194,90.983000,67.420,Culvert,Box culvert,23.495139,90.982999,CorrectData
69,70.0,Chandinan-Shahapara Bridge.,C,9.20,22.55,1986.0,3.0,Comilla,Comilla,Comilla,...,3085.0,71.156,23.489111,91.012056,70.527,Bridge,Bridge start,23.489055,91.012028,CorrectData
70,71.0,CORI PAI BOX CULVERT,B,20.00,4.60,1999.0,1.0,Comilla,Comilla,Comilla,...,1181.0,74.279,23.481278,91.041139,73.688,Culvert,Box culvert,23.481166,91.040444,CorrectData
71,72.0,COREPAI BOX CULVERT,B,20.00,4.60,1991.0,1.0,Comilla,Comilla,Comilla,...,2214.0,75.312,23.482083,91.050444,74.700,Culvert,Box culvert,23.482083,91.050417,CorrectData
73,74.0,KABILA DUBARCHAR SLAB CULVERT,C,12.30,1.00,1993.0,1.0,Comilla,Comilla,Comilla,...,1321.0,78.382,23.477028,91.079278,77.773,Culvert,Box culvert,23.477083,91.078999,CorrectData
75,76.0,TIPRA BAZAR BOX CULVERT,A,12.20,1.40,1993.0,1.0,Comilla,Comilla,Comilla,...,3363.0,82.437,23.478833,91.117750,81.801,Culvert,Box culvert,23.478944,91.117722,CorrectData
640,640.0,Brorechon,A,9.70,1.50,2000.0,1.0,Comilla,Comilla,Comilla,...,58.0,2.059,23.493750,91.106472,2.064,Culvert,Box Culvert,23.493777,91.106472,CorrectData


In [79]:
FinalData.to_csv('TinderFile.csv')